In [ ]:
!pip install mlflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, tree
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow

In [ ]:
iris = datasets.load_iris()

In [ ]:
X = pd.DataFrame(iris.data, columns=iris.feature_names)
X.columns = ["sepal_length","sepal_width","petal_length","petal_width"]
X.head()

In [ ]:
y = pd.DataFrame(iris.target, columns=["target"])
y.head()

In [ ]:
XTrain, XTest, yTrain, yTest = train_test_split(X,y,test_size=0.33,random_state=1)

In [ ]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
for n in range (50, 151, 5):
    with mlflow.start_run():
        num_estimators = n
        mlflow.log_param("num_estimators", num_estimators)

        rf = RandomForestClassifier(n_estimators=num_estimators)
        rf.fit(XTrain, yTrain)

        predicted_qualities = rf.predict(XTest)

        (rmse, mae, r2) = eval_metrics(yTest, predicted_qualities)
  
        print("Random Forest Classifier (n_estimators=%d):" % (num_estimators))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
        if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
          mlflow.sklearn.log_model(rf, "model", registered_model_name="RandomForest")
        else:
          mlflow.sklearn.log_model(rf, "model")